In [61]:
import pandas as pd
import numpy as np
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

<IPython.core.display.Javascript object>

In [62]:
properati = pd.read_csv('datos/new.csv',index_col=0,error_bad_lines=False)
properati.columns

Index([u'created_on', u'operation', u'property_type', u'place_name',
       u'place_with_parent_names', u'country_name', u'state_name',
       u'geonames_id', u'lat-lon', u'lat', u'lon', u'price', u'currency',
       u'price_aprox_local_currency', u'price_aprox_usd',
       u'surface_total_in_m2', u'surface_covered_in_m2', u'price_usd_per_m2',
       u'price_per_m2', u'floor', u'rooms', u'expenses', u'properati_url',
       u'description', u'title', u'image_thumbnail'],
      dtype='object')

In [63]:
print len(properati)
nuevo = properati.loc[(properati.state_name.str.contains("Capital Federal")) | (properati.state_name.str.contains("G.B.A"))]
print len(nuevo)

1463039
991548


In [64]:
%%notify
nuevo = nuevo.loc[nuevo['price']!= 0]
nuevo = nuevo.dropna(subset=['price'])
nuevo.drop_duplicates(inplace=True)
nuevo.drop_duplicates(subset=['properati_url'],inplace=True)

print len(nuevo)
conv_dolar_ars= 17.6445 #precio del dolar usado por properati

def recalcular_precios(row):
    if (row.currency == 'USD'):
        row.price_aprox_usd   = float(row.price)
        row.price_aprox_local_currency  = row.price_aprox_usd * conv_dolar_ars # pasa usd a ars
        
    if (row.currency == 'ARS'):
        row.price_aprox_local_currency  = float(row.price)
        row.price_aprox_usd  = row.price_aprox_local_currency // conv_dolar_ars # pasa ars a usd
        
    if (row.surface_total_in_m2 and row.price_aprox_local_currency and row.price_aprox_usd): #si tiene la sup total y los precios calcular los precios por m2
        row.price_per_m2   = row.price_aprox_local_currency  // row.surface_total_in_m2
        row.price_usd_per_m2  = row.price_aprox_usd // row.surface_total_in_m2
     
    if ( np.isnan(row.surface_total_in_m2) and row.surface_covered_in_m2 ): #si no tiene la sup total usa la descubierta como total 
        row.surface_total_in_m2 = row.surface_covered_in_m2
    
    if (row.price_per_m2 and (np.isnan(row.price_usd_per_m2))): 
        row.price_usd_per_m2  = row.price_per_m2 // conv_dolar_ars
        
    if ((np.isnan(row.price_per_m2)) and (np.isnan(row.price_usd_per_m2)) and (not(np.isnan(row.surface_total_in_m2)) ) ):
        row.price_usd_per_m2 = row.price_aprox_usd * row.surface_total_in_m2
        row.price_per_m2 = row.price_aprox_local_currency * row.surface_total_in_m2
        
    return row
nuevo = nuevo.apply(recalcular_precios,axis=1)

202581


<IPython.core.display.Javascript object>

In [65]:
total= len(nuevo)
print total
print len (nuevo.dropna(subset=['price_usd_per_m2']))
print len (nuevo.dropna(subset=['price_per_m2']))
print len (nuevo.dropna(subset=['surface_total_in_m2']))
print len (nuevo.dropna(subset=['surface_covered_in_m2']))
print len (nuevo.dropna(subset=['price_usd_per_m2','price_per_m2','surface_total_in_m2','surface_covered_in_m2']))

202581
189836
189836
189836
182920
182916


In [66]:
nuevo =nuevo.dropna(subset=['price_usd_per_m2','price_per_m2','surface_total_in_m2','surface_covered_in_m2','place_name'])

In [67]:
print len(nuevo)

182872


In [68]:
nuevo.to_csv('datos/properati_2.csv',index=False)